Go up one level to access all of the scripts from the main director

In [2]:
import os
os.chdir('..')

Create some simple X, y training values

In [66]:
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)

from solar.wrangle.wrangle import SolarData
from solar.wrangle.subset import Subset
station = ['ACME', 'BEAV']
train_dates = ['2005-07-29','2005-08-29']
test_dates = ['2008-12-29', '2009-01-29']
#var = ['dswrf_sfc', 'uswrf_sfc', 'spfh_2m', 'ulwrf_tatm']
var = ['dswrf_sfc']
models = [0]
times = [18]
lats = range(37,38)
longs = range(265,268)
input_data = SolarData.load('solar/data/kaggle_solar/train/', 'solar/data/kaggle_solar/train.csv', 
                                'solar/data/kaggle_solar/test', 'solar/data/kaggle_solar/station_info.csv',
                                var=var, lats=lats, longs=longs, test_dates=test_dates, train_dates=train_dates,
                                models=models, times=times, station=station)

Training data shape (32, 3) (32, 2)


In [67]:
trainX = input_data[0]
trainy = input_data[1]
trainy

location,ACME,BEAV
date,,
2005-07-29,28302900,29221200
2005-07-30,26890800,27926700
2005-07-31,26932200,27732300
2005-08-01,26653800,27705600
2005-08-02,26087700,27396000
2005-08-03,26110200,26341200
2005-08-04,21778200,13657500
2005-08-05,23165700,21474300
2005-08-06,16723200,21845400


In [37]:
import solar.analyze.model
from solar.analyze.model import Model

In [116]:
reload(solar.analyze.model)
from solar.analyze.model import Model

from sklearn.linear_model import Ridge
from sklearn import metrics

error_formula = 'mean_absolute_error'

trainX, trainy, model = Model.model(input_data, Ridge, {'alpha':[0.2,0.3]}, 10, 
                    error_formula, normalize=True)

In [120]:
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
trainX, trainy, model = Model.model(input_data, KNeighborsRegressor, 
                                    {'n_neighbors':np.arange(1,10)}, 10, 
                                    error_formula)

In [121]:
model.fit(trainX,trainy)

GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_neighbors': array([1, 2, 3, 4, 5, 6, 7, 8, 9])},
       pre_dispatch='2*n_jobs', refit=True, scoring='mean_absolute_error',
       verbose=0)

In [126]:
model.best_score_

-5067979.166666667

In [123]:
model.predict(trainX.iloc[20:21,:])
print trainX.shape
print trainy.shape
print model.predict(trainX.iloc[0:1,:])

(32, 3)
(32, 2)
[[ 24152833.33333333  25498633.33333333]]


In [132]:
import datetime
strftime("%Y%m%d-%H%M%S")

'2016-01-18-17-15-21'

In [223]:
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)

from solar.wrangle.wrangle import SolarData
#station = ['ACME', 'BEAV']
station = ['all']
#train_dates = ['2005-07-29','2005-08-29']
train_dates = []
#test_dates = ['2008-12-29', '2009-01-29']
test_dates = []
#var = ['dswrf_sfc', 'uswrf_sfc', 'spfh_2m', 'ulwrf_tatm']
var = ['dswrf_sfc']
models = [0]
#times = [18]
times = []
#lats = range(37,38)
lats = []
#longs = range(265,268)
longs = []
trainX, trainy, testX = SolarData.load('solar/data/kaggle_solar/train/', 'solar/data/kaggle_solar/train.csv', 
                                'solar/data/kaggle_solar/test', 'solar/data/kaggle_solar/station_info.csv',
                                var=var, lats=lats, longs=longs, test_dates=test_dates, train_dates=train_dates,
                                models=models, times=times, station=station)

Training data shape (5113, 720) (5113, 98)


In [224]:
reload(solar.analyze.model)
from solar.analyze.model import Model

from sklearn.linear_model import Ridge
from sklearn import metrics

error_formula = 'mean_absolute_error'

model = Model.model([trainX,trainy,testX], Ridge, {'alpha':np.logspace(-3,1,8,base=10)}, 10, 
                    error_formula, normalize=True)

In [225]:
reload(solar.report.submission)
from solar.report.submission import Submission

preds = Submission.make_submission_file(model, trainy, testX)

In [221]:
preds.shape

(1796, 98)

In [226]:
model.best_score_

-2547805.8119099629

In [227]:
import cPickle as pickle

In [229]:
model = pickle.load(open('solar/data/kaggle_solar/model.p','rb'))

In [232]:
model.grid_scores_

[mean: -6810007.57575, std: 338020.07264, params: {'alpha': 0.001},
 mean: -6793310.06300, std: 339836.63444, params: {'alpha': 0.0037275937203149379},
 mean: -6787378.97584, std: 341423.87630, params: {'alpha': 0.013894954943731374},
 mean: -6786606.73713, std: 342246.64587, params: {'alpha': 0.0517947467923121},
 mean: -6786692.36930, std: 342599.88944, params: {'alpha': 0.19306977288832497},
 mean: -6786762.10570, std: 342720.09553, params: {'alpha': 0.71968567300115138},
 mean: -6786772.49729, std: 342754.73196, params: {'alpha': 2.6826957952797246},
 mean: -6786723.58849, std: 342763.51199, params: {'alpha': 10.0}]